## Marker Selection

In [4]:
import pathlib
from concurrent.futures import ProcessPoolExecutor, as_completed
from itertools import combinations
import numpy as np
import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns
import xarray as xr
import subprocess
from sklearn.metrics import roc_auc_score
from cemba_data.tools.hdf5.anndata import rank_features_groups
import warnings
warnings.filterwarnings('ignore')

## Parameter

In [5]:
mcds_pattern = '/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.*.mcds'
min_cluster_cell_number = 10
adj_p_cutoff = 1e-1
top_n = 1000
max_test_cell_population = 100
chunk_size = 1
delta_rate_cutoff = 0.3
auroc_cutoff = 0.8

In [6]:
# Parameters
cluster_col = "SubType"
use_clusters = [
    "MGE-Sst Rxra", "CA3 Cadm2", "CA1 Chrm3", "CA3-St18 Tead1", "Unc5c Unc5c",
    "Gfra1 Gfra1", "IT-L5 Etv1", "CA1 Ptprg", "MGE-Sst Ptpre", "NP-L6 Cntnap4",
    "CA3-St18 Nuak1", "CGE-Lamp5 Dock5", "CT-L6 Megf9", "IG-CA2 Chrm3",
    "IG-CA2 Peak1", "DG-po Calb2", "DG dg-all", "CGE-Vip Ntng1", "CA1 Kif26a",
    "CA3 Efnb2", "CGE-Vip Ptprm", "CA1 Ak5", "DG-po Bcl11a", "PT-L5 Tenm2",
    "CGE-Vip Robo1", "CA1 Lingo2", "MGE-Pvalb Gfra2", "CA3-St18 Epha5",
    "PAL-Inh Meis2", "IG-CA2 Xpr1", "MGE-Sst Unc5b", "MGE-Pvalb Thsd7a",
    "CGE-Vip Grm8", "MGE-Sst Dock4", "CGE-Lamp5 Grk5", "OLF Xkr6",
    "DG-po Kctd8", "MSN-D2 Slc24a2", "CGE-Lamp5 Sorcs1", "CT-L6 Il1rap",
    "L6b Adcy8", "MGE-Pvalb Entpd3", "IT-L6 Man1c1", "MGE-Pvalb Ptprk",
    "CGE-Vip Ccser1", "NP-L6 Olfml2b", "CGE-Lamp5 Grid1", "MGE-Pvalb Sema5a",
    "MGE-Sst Kcnip4", "PT-L5 Abca12", "MGE-Sst Frmd6", "MGE-Pvalb Cnih3",
    "MGE-Sst Ubtd1", "PT-L5 Nectin1", "MGE-Sst Rerg", "CGE-Vip Fstl4",
    "CGE-Vip Galnt17", "MGE-Sst Etv1", "IT-L23 Cux1", "IT-L23 Foxp1",
    "CGE-Vip Clstn2", "IT-L4 Shc3", "IT-L5 Cdh8", "IT-L5 Grik3", "PT-L5 Tmtc2",
    "IT-L23 Tenm2", "NP-L6 Cntnap5a", "CT-L6 Hcrtr2", "PT-L5 Plcb4",
    "IT-L23 Ptprt", "CGE-Lamp5 Nrxn3", "CT-L6 Map4", "MGE-Sst Chodl",
    "NP-L6 Boc", "PT-L5 Kcnh1", "OLF-Exc Bmpr1b", "OLF Trpc4", "PT-L5 Astn2",
    "IT-L6 Fstl4", "CLA Bcl11a", "NP-L6 Cyp7b1", "CLA Cdh8", "IT-L6 Cadps2",
    "PT-L5 Ptprt", "NP-L6 Kcnab1", "IT-L6 Oxr1", "Foxp2 Homer2",
    "MGE-Pvalb Cacna1i", "MSN-D1 Khdrbs3", "MSN-D1 Plxnc1", "OLF Mapk10",
    "MSN-D1 Hrh1", "Foxp2 Trpc7", "OLF Pag1", "MSN-D2 Col14a1",
    "MGE-Sst Bmper", "OLF-Exc Pld5", "OLF Gabbr2", "OLF Kcnd3",
    "PAL-Inh Deptor", "OLF-Exc Lrrtm3", "OLF-Exc Cdh9", "OLF-Exc Unc13c",
    "PAL-Inh Meis1", "L6b Nrp2", "LSX-Inh Cacna1i", "OLF-Exc Sgcd",
    "OLF-Exc Rmst", "PT-L5 Unc5b", "L6b Pkhd1", "L6b Kcnk2", "IT-L4 Astn2",
    "CLA Nrp2", "D1L-Fstl4 Sipa1l2", "EP Tspan5", "PAL-Inh Rarb",
    "MSN-D2 Nrp2", "D1L-Fstl4 Trps1", "Foxp2 Dchs2", "OLF-Exc Cux2",
    "PAL-Inh Chat", "D1L-PAL Flrt2", "EP Rgs8", "PAL-Inh Igdcc3",
    "PAL-Inh Tmem178", "MSN-D1 Ntn1", "Foxp2 Inpp4b", "MSN-D2 Casz1",
    "Chd7 Kcnc2", "PAL-Inh Tcf7l2", "D1L-Fstl4 Grm3", "D1L-Fstl4 Cadm1",
    "Chd7 Trpc7", "PAL-Inh Ptprd", "D1L-Fstl4 Crim1", "Chd7 Megf11",
    "EP Adcy8", "D1L-PAL Plcxd3", "PAL-Inh Onecut2", "LSX-Inh Foxp2",
    "LSX-Inh Enox1", "LSX-Inh Dock10", "LSX-Inh Nxph1", "LSX-Inh Zeb2",
    "LSX-Inh Lats2"
]
mcds_pattern = "/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.*.mcds"
min_cluster_cell_number = 10
adj_p_cutoff = 0.005
top_n = 1000
cpu = 20
max_test_cell_population = 1000
chunk_size = 50
delta_rate_cutoff = 0.3
auroc_cutoff = 0.8

### Stable Parameter

In [7]:
tidy_data_path = '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
random_seed = 0

## Cell Tidy Data

In [8]:
cell_tidy_data = pd.read_msgpack(tidy_data_path)

if use_clusters is not None:
    cell_tidy_data = cell_tidy_data[cell_tidy_data[cluster_col].isin(
        use_clusters)]
cell_tidy_data.shape[0]

95401

In [9]:
records = []
for cluster, sub_df in cell_tidy_data.groupby(cluster_col):
    if sub_df.shape[0] < max_test_cell_population:
        records.append(sub_df)
    else:
        records.append(
            sub_df.sample(max_test_cell_population, random_state=random_seed))
cell_tidy_data = pd.concat(records)
cell_tidy_data[cluster_col].value_counts()

IT-L23 Ptprt      1000
IT-L5 Cdh8        1000
CA1 Chrm3         1000
OLF-Exc Sgcd      1000
MSN-D2 Slc24a2    1000
                  ... 
CA1 Lingo2          54
DG-po Calb2         51
OLF Xkr6            46
DG-po Kctd8         39
L6b Pkhd1           37
Name: SubType, Length: 145, dtype: int64

In [10]:
cluster_series = cell_tidy_data[cluster_col].astype(str)
cluster_counts = cluster_series.value_counts()


def check_cluster(cluster, count):
    if count < min_cluster_cell_number:
        return False
    return True


unique_clusters = [
    cluster for cluster, count in cluster_counts.items()
    if check_cluster(cluster, count)
]
cluster_pairs = list(combinations(unique_clusters, 2))

print(len(unique_clusters), 'pass filter.')
print(len(cluster_pairs), 'pairwise comparison to test.')

145 pass filter.
10440 pairwise comparison to test.


## Gene meta

In [11]:
gene_meta = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusterMethylMarker/gencode.vM22.annotation.gene.flat.filtered_white_genes.tsv.gz',
    index_col='gene_id',
    sep='\t')
gene_meta.index.name = 'gene'
gene_name_to_id = {v: k for k, v in gene_meta['gene_name'].iteritems()}
gene_idbase_to_id = {i.split('.')[0]: i for i in gene_meta.index}

## Adata

In [9]:
gene_mcds = xr.open_mfdataset(mcds_pattern)
use_gene = gene_mcds.get_index('gene') & gene_meta.index
gene_meta = gene_meta.reindex(use_gene)

In [10]:
gene_mcds

<xarray.Dataset>
Dimensions:           (cell: 104340, gene: 55487, mc_type: 2)
Coordinates:
  * mc_type           (mc_type) object 'CGN' 'CHN'
    strand_type       <U4 'both'
  * gene              (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
    geneslop2k_chrom  (gene) object 'chr1' 'chr1' 'chr1' ... 'chrM' 'chrM'
    geneslop2k_start  (gene) int64 3071252 3100015 3203900 ... 12144 13288 13355
    geneslop2k_end    (gene) int64 3076321 3104124 3673497 ... 16299 16299 16299
  * cell              (cell) object '1A_M_0' '1A_M_1' ... '8J_M_1292'
Data variables:
    gene_da           (cell, gene, mc_type) float64 dask.array<shape=(104340, 55487, 2), chunksize=(10000, 55487, 2)>

In [11]:
gene_mcds = gene_mcds['gene_da'].sel(mc_type='CHN',
                                     cell=cell_tidy_data.index,
                                     gene=gene_meta.index)
gene_mcds

<xarray.DataArray 'gene_da' (cell: 59714, gene: 50231)>
dask.array<shape=(59714, 50231), dtype=float64, chunksize=(122, 50231)>
Coordinates:
    mc_type           <U3 'CHN'
    strand_type       <U4 'both'
  * gene              (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
    geneslop2k_chrom  (gene) object 'chr1' 'chr1' 'chr1' ... 'chrM' 'chrM'
    geneslop2k_start  (gene) int64 3071252 3100015 3203900 ... 12144 13288 13355
    geneslop2k_end    (gene) int64 3076321 3104124 3673497 ... 16299 16299 16299
  * cell              (cell) object '10E_M_1566' '10E_M_1589' ... '9J_M_572'

In [12]:
cell_tidy_data.to_msgpack('TEMP.msg')

In [13]:
gene_mcds.to_netcdf('TEMP.nc')

## Pairwise test

In [9]:
def get_sig_features(rank_gene_dict):
    pvals_adj = pd.DataFrame(rank_gene_dict['pvals_adj'])
    names = pd.DataFrame(rank_gene_dict['names'])
    return pvals_adj, names


def get_delta(cluster, gene):
    row = cluster_mean_gene_df.loc[gene].copy()
    cluster_value = row.pop(cluster)
    other_mean = row[0] # only two cluster
    delta = cluster_value - other_mean
    return delta


def calculate_single_pair(data_path, pair):
    cluster_a, cluster_b = pair
    output_dir = pathlib.Path(f'TEMP/{cluster_a}')
    output_dir.mkdir(exist_ok=True)
    output_path = output_dir / f'{cluster_b}.msg'
    if output_path.exists():
        return
    
    this_tidy_data = pd.read_msgpack('TEMP.msg')
    this_tidy_data = this_tidy_data[this_tidy_data[cluster_col].isin(pair)]
    
    mcds = xr.open_dataarray(data_path).sel(cell=this_tidy_data.index).load()
    adata = anndata.AnnData(X=mcds.values,
                    obs=pd.DataFrame([], mcds.get_index('cell')),
                    var=pd.DataFrame([], mcds.get_index('gene')))
    adata.obs['cluster'] = this_tidy_data[cluster_col].astype('category')
    
    # reverse_adata, centered by 1 because after normalization all prior is center to 1
    adata.X = (adata.X - 1) * -1 + 1
    
    # calculate cluster delta
    records = {}
    for cluster, sub_df in adata.obs.groupby('cluster'):
        sub_adata = adata[sub_df.index, :]
        gene_mean = sub_adata.X.mean(axis=0)
        records[cluster] = pd.Series(gene_mean, index=sub_adata.var_names)
    cluster_mean_gene_df = pd.DataFrame(records)

    # judge gene by delta
    cluster_delta = (cluster_mean_gene_df[cluster_a] -
                     cluster_mean_gene_df[cluster_b]).abs()
    delta_judge = cluster_delta > delta_rate_cutoff
    use_adata = adata[:, delta_judge]
    
    # sc.tl.rank_genes_groups will do expm1 internally when calculate fold change...
    # do not use the fold change calculated by sc.tl.rank_genes_groups
    sc.tl.rank_genes_groups(use_adata,
                            groupby='cluster',
                            n_genes=top_n,
                            method='wilcoxon')
    pvals_adj, names = get_sig_features(
        use_adata.uns['rank_genes_groups'])

    results = []
    for col in use_adata.obs['cluster'].unique():
        if col not in pair:
            continue
        df = pd.DataFrame({
            'pvals_adj': pvals_adj[col].tolist(),
            'gene_id': names[col].tolist()
        })
        df['cluster_from'] = col
        df['cluster_to'] = cluster_a if col == cluster_b else cluster_b
        results.append(df)
    
    # get total results filter by adj_p
    total_results = pd.concat(results)
    total_results['gene_name'] = total_results['gene_id'].map(
        gene_meta['gene_name'])
    total_results['-lgp'] = -np.log10(total_results['pvals_adj'])
    total_results['-lgp'] = total_results['-lgp'].replace(np.inf, 1000)
    total_results = total_results[
        total_results['pvals_adj'] < adj_p_cutoff].copy()

    # judge by auroc
    total_results['AUROC'] = total_results[[
        'gene_id', 'cluster_from'
    ]].apply(lambda i: get_auroc(i['gene_id'], i['cluster_from'],
                                 use_adata),
             axis=1)
    total_results = total_results[total_results['AUROC'] > auroc_cutoff]
    total_results.to_msgpack(output_path)
    return


def get_auroc(gene_id, cluster, adata):
    yscore = adata.obs_vector(gene_id)
    ylabel = adata.obs['cluster'] == cluster
    score = roc_auc_score(ylabel, yscore)
    score = abs(score - 0.5) + 0.5
    return score

In [10]:
data_path = 'TEMP.nc'

## Run pairwise marker

In [ ]:
pair_marker_counts = {}
n = 0
chunk_size = cpu
for chunk_start in range(0, len(cluster_pairs), chunk_size):
    pairs = cluster_pairs[chunk_start : chunk_start + chunk_size]
    
    with ProcessPoolExecutor(cpu) as executor:
        temp_dir = 'TEMP'
        pathlib.Path(temp_dir).mkdir(exist_ok=True)
        futures = []
        for pair in pairs:
            future = executor.submit(calculate_single_pair, data_path, pair)
            futures.append(future)
    
        for future in as_completed(futures):
            n += 1
            if n % 100 == 0:
                print(n)
            future.result()

100
200
300
400
500
600
700


## Aggregate DEG

In [12]:
temp_dir = pathlib.Path('TEMP')
deg_list = list(temp_dir.glob('**/*msg'))

df_list = []
for path in deg_list:
    df_list.append(pd.read_msgpack(path))

total_markers = pd.concat(df_list)

In [13]:
assert total_markers.set_index(['gene_id', 'cluster_from', 'cluster_to']).index.duplicated().sum() == 0

In [14]:
total_markers.shape

(18477109, 7)

## Save results

In [15]:
total_markers.to_msgpack('TotalPairwiseMarker.msg')

In [16]:
marker_counts = total_markers.set_index(['cluster_from', 'cluster_to']).index.value_counts()
marker_counts.index = pd.MultiIndex.from_tuples(marker_counts.index.tolist())
marker_counts = marker_counts.reset_index()
marker_counts.columns = ['ClusterA', 'ClusterB', 'GeneCount']
marker_counts.to_csv('Cluster_pair_marker_counts.csv', index=None)

In [17]:
marker_counts[marker_counts['GeneCount'] < 10]

,ClusterA,ClusterB,GeneCount
20878,CT-L6 Il1rap,CT-L6 Megf9,7
20879,CT-L6 Megf9,CT-L6 Il1rap,7


In [29]:
subprocess.run('rm -rf TEMP*', shell=True)

CompletedProcess(args='rm -rf TEMP*', returncode=0)

In [18]:
with open('TotalGeneID.txt', 'w') as f:
    for g in total_markers['gene_id'].unique():
        f.write(f'{g}\n')

In [19]:
total_markers['gene_id'].unique().size

27194

In [21]:
_marker_counts = total_markers[total_markers['AUROC'] > 0.85].set_index(
    ['cluster_from', 'cluster_to']).index.value_counts()
_marker_counts.index = pd.MultiIndex.from_tuples(_marker_counts.index.tolist())
_marker_counts = _marker_counts.reset_index()
_marker_counts.columns = ['ClusterA', 'ClusterB', 'GeneCount']
_marker_counts[_marker_counts['GeneCount'] < 10]


,ClusterA,ClusterB,GeneCount
20869,LSX-Inh Nxph1,LSX-Inh Dock10,9
20870,MSN-D2 Slc24a2,MSN-D2 Nrp2,9
20871,MSN-D1 Plxnc1,MSN-D1 Hrh1,8
20872,PT-L5 Plcb4,PT-L5 Ptprt,8
20873,PT-L5 Ptprt,PT-L5 Plcb4,8
20874,D1L-PAL Plcxd3,D1L-PAL Flrt2,8
20875,OLF-Exc Rmst,OLF-Exc Cdh9,6
20876,LSX-Inh Dock10,PAL-Inh Meis2,4
20877,CT-L6 Il1rap,CT-L6 Megf9,2
20878,CT-L6 Megf9,CT-L6 Il1rap,2


In [23]:
total_markers[(total_markers['cluster_from'] == 'PAL-Inh Meis2') & 
              (total_markers['cluster_to'] == 'PAL-Inh Igdcc3')]

,pvals_adj,gene_id,cluster_from,cluster_to,gene_name,-lgp,AUROC
0,7.120809e-107,ENSMUSG00000054966.13,PAL-Inh Meis2,PAL-Inh Igdcc3,Lmntd1,106.147471,0.876676
1,5.826419e-86,ENSMUSG00000045659.18,PAL-Inh Meis2,PAL-Inh Igdcc3,Plekha7,85.234598,0.836973
2,6.477870e-86,ENSMUSG00000027210.20,PAL-Inh Meis2,PAL-Inh Igdcc3,Meis2,85.188568,0.836848
3,5.841211e-83,ENSMUSG00000004110.17,PAL-Inh Meis2,PAL-Inh Igdcc3,Cacna1e,82.233497,0.830921
4,6.150232e-75,ENSMUSG00000032511.17,PAL-Inh Meis2,PAL-Inh Igdcc3,Scn5a,74.211108,0.814104
5,1.852673e-73,ENSMUSG00000040612.13,PAL-Inh Meis2,PAL-Inh Igdcc3,Ildr2,72.732201,0.810864
6,4.805615e-73,ENSMUSG00000025372.16,PAL-Inh Meis2,PAL-Inh Igdcc3,Baiap2,72.318251,0.809961
7,6.227768e-73,ENSMUSG00000010755.17,PAL-Inh Meis2,PAL-Inh Igdcc3,Cars,72.205668,0.809700
8,8.564660e-69,ENSMUSG00000039410.16,PAL-Inh Meis2,PAL-Inh Igdcc3,Prdm16,68.067290,0.800494
9,1.266042e-68,ENSMUSG00000025812.17,PAL-Inh Meis2,PAL-Inh Igdcc3,Pard3,67.897552,0.800088


In [2]:
total_markers = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusterMethylMarker/SubTypePairwiseDEG/TotalPairwiseMarker.msg'
)

output_dir = 'PairwiseMarkerPerCluster'
pathlib.Path(output_dir).mkdir(exist_ok=True)

for cluster, sub_df in total_markers.groupby('cluster_from'):
    print(cluster)
    sub_df.to_msgpack(f'{output_dir}/{cluster}.msg')